# Wetterskip Fryslan
### Onderscheid water van niet-water obv RGB image en Class image

#### Idee
- gebruik een sliding window met variabel formaat
- zet class image (zwart wit foto) om naar een vector. Het resultaat is dus 1 lange rij met eenen (water) en nullen (geen water).
- zet feature image (kleuren foto) om naar een matrix met evenveel rijen als de class vector. Deze feature matrix bevat uiteindelijk in elke rij, per layer (RGB) het aantal pixels van het sliding window. Dus stel het window is 2 bij 2 en de RGB foto heeft 3 lagen dan heeft elke rij in de matrix 2 maal 2 maal 3 = 12 kolommen. 

#### Dit script
- Data preparatie: 
    - lees class en feature images in array
    - Met sliding windows techniek om feature en class maken
- Splitsen data in een train en testset (validatie wordt in ander script gedaan)
- Train model op :
    - Naive Bayes
    - Logistic Regression
    - Random Forrest

#### Todo
- Labelling afmaken
    - Basis opzet is klaar
    - class '2' verwijderen uit test en train set. Dit betreft categorie 'onbekend'
    - 180418 OD: Categorie 'onbekend' verwijderd. Indien threshold waarde voor water niet wordt gehaald, dan 'niet water'=0
- Koppelen aan werkelijke (RGB) foto  
    - Basis opzet is klaar
    - 180418 OD: originele tiff sample bestanden kunnen niet worden ingelezen met PIL omdat deze een afwijkende encoding hebben. Daarom deze met image editor omgezet naar .png.
- Test/train set maken
    - Basis opzet is klaar
    - 180418 OD: nu nog nieuwe foto als validatie set gebruiken      
- Model kiezen 
    - Eerste test met logistic regression gedaan  
    - Naive base classificatie als baseline ingevoegd)  
    - RandomForrestClassifier geeft betere score op testset.  
    - Na iedere fit, met pickle het getrainde model opslaan voor nieuwe trainingsdoeleinden.      
- Parameters tunen

#### Opmerkingen
- Paul: ik heb nog wat ruzie met het openen van de tif bestanden. Om die reden omgezet naar jpg. Bij voorkeur wordt dit nog aangepast.  
- 180418 OD: jpg geen favoriet formaat ivm lossy compression. PNG is een goed alternatief voor TIF

## Bibliotheken laden

In [1]:
import numpy as np
import pickle
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB

## Open Class Bestand

In [2]:
# open class bestand in ZWART-WIT (convert('l'))
ClassImg = Image.open("labels.tif").convert('L')
# zet om naar array (matrix)
ClassArray = np.array(ClassImg)
# zet breedte en hoogte om in variabelen
ClassImageWidth, ClassImageHeight = ClassArray.shape

In [3]:
ClassArray

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255]], dtype=uint8)

In [4]:
# vervang alle waarden van 255 (wit) door 1 en alle andere door 0 (zwart)
ClassArray[ClassArray < 255] = 1 # zwart/water
ClassArray[ClassArray == 255] = 0 # wit/geen water

In [5]:
# grootte van het sliding window
WindowLength = 5
WindowSize = WindowLength**2
# variable ter bepaling van de hoeveelheid pixels naar rechts en naar beneden (de 'slide')
Step = 1 
# tijdelijke teller
Counter = 0 
# drempel waarboven een window als water wordt geclassificeerd
Threshold = .8 

# bepaal hoe groot de output vector wordt en creeer een lege array:
ClassArrayResult = np.zeros(shape=(((ClassImageWidth - WindowLength) * (ClassImageHeight - WindowLength))))

# voor elke rij
for i in range(0,ClassImageWidth-WindowLength,Step):
    # voor elke kolom
    for j in range(0, ClassImageHeight-WindowLength,Step):
        # wanneer de som van de waarden in het window gelijk is aan het totale aantal pixels in het window dan
        # zitten er dus alleen maar 1'en in --> dus water
        if (ClassArray[i:i + WindowLength, j:j + WindowLength]==1).sum() == (WindowSize):
            ClassArrayResult[Counter] = 1
        # wanneer de som van de waarden in het window gelijk is aan nul dan zitten er dus geen 1'en in --> dus geen water
        elif (ClassArray[i:i + WindowLength, j:j + WindowLength]==1).sum() == 0:
            ClassArrayResult[Counter] = 0
        # wanneer het aantal 1'en gedeeld door het aantal pixels groter is dan de toegestane drempel, dan alsnog 
        # classificeren als 'water'
        elif ((ClassArray[i:i + WindowLength, j:j + WindowLength]==1).sum() / (WindowSize)) > Threshold:
            ClassArrayResult[Counter] = 1
        # in de resterende gevallen is het onbekend
        else:
            ClassArrayResult[Counter] = 0
            
        Counter += 1

## Openen Feature foto (RGB)

In [6]:
# open feature foto (RGB).
FeatureImg = Image.open("feature1.png").convert('RGB')
# zet om naar array (matrix)
FeatureArray = np.array(FeatureImg)
# zet breedte, hoogte en diepte om in variabelen
FeatureImageWidth, FeatureImageHeight, FeatureImageLayers = FeatureArray.shape

In [7]:
# bepaal het formaat van de output matrix en creeer een lege array:
FeatureArrayResult = np.zeros(shape=(((FeatureImageWidth - WindowLength) * (FeatureImageHeight - WindowLength)), (WindowSize)*FeatureImageLayers))

# voor elke layer
for z in range(0, FeatureImageLayers):
    Counter = 0
    # voor elke rij
    for i in range(0, FeatureImageWidth-WindowLength, Step):
        # voor elke kolom
        for j in range(0, FeatureImageHeight-WindowLength, Step):
            # zet de betreffende waarden per layer in feature matrix
            FeatureArrayResult[Counter, (z*(WindowSize)):(z*(WindowSize)+(WindowSize))] = (FeatureArray[i:i + WindowLength, j:j + WindowLength, z]).flatten()          
            Counter += 1


## Splitsen data in Train- en Testset

In [8]:
# maak train en test set
X_train, X_test, y_train, y_test = train_test_split(FeatureArrayResult, ClassArrayResult, random_state=0) 
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## Naive Bayes (baseline)

### Train model met Naive Bayes

In [9]:
# train een model op basis van Gaussian Naive Bayes
gnb = GaussianNB()
gnb_clf = gnb.fit(X_train,y_train)

In [10]:
# save het getrainde Naive Bayes model naar schijf
filename = 'GaussianNaiveBayes_model.sav'
pickle.dump(gnb_clf, open(filename, 'wb'))

### Voorspel met Gaussian Naive Bayes

In [11]:
# bekijk resultaten
gnb_predictions = gnb_clf.predict(X_test)
print(classification_report(y_test, gnb_predictions))

             precision    recall  f1-score   support

        0.0       0.94      0.87      0.90    183679
        1.0       0.69      0.83      0.75     63828

avg / total       0.87      0.86      0.86    247507



## Logistic Regression

### Train Logistic Regression model

In [12]:
# train model (Logistic regression in dit geval, eerst zonder parameter tuning)
classifier = LogisticRegression()    
classifier.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
# save het getrainde LogisticRegression model naar schijf
filename = 'Logistic_Regression_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

### Voorspelling Logistic Regression model

In [14]:
# bekijk resultaten
predictions = classifier.predict(X_test)
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

        0.0       0.98      0.98      0.98    183679
        1.0       0.94      0.94      0.94     63828

avg / total       0.97      0.97      0.97    247507



## Random Forrest

### Train Random Forrest model

In [15]:
# train model RandomForrest 
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [16]:
filename = 'RandomForrestClassifier_model.sav'
pickle.dump(rf_clf, open(filename, 'wb'))

### Voorspel uitkomt op basis van Random Forrest

In [17]:
rf_pred = rf_clf.predict(X_test)
print(classification_report(y_test, rf_pred))

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99    183679
        1.0       0.99      0.97      0.98     63828

avg / total       0.99      0.99      0.99    247507

